# First example of GAN's Networks

Based on this medium article: https://medium.com/@devnag/generative-adversarial-networks-gans-in-50-lines-of-code-pytorch-e81b79659e3f

# To-do list:

* () - Ver o por quê de ser tantas épocas;

* () - Ver a questão da frequência de treinamento;

* () - Ver a questão do detach();


# Notebook


There are really only 5 components to think about:

R: The original, genuine data set

I: The random noise that goes into the generator as a source of entropy

G: The generator which tries to copy/mimic the original data set

D: The discriminator which tries to tell apart G’s output from R

The actual ‘training’ loop where we teach G to trick D and D to beware G.


In [90]:
import torch as tc
import numpy as np
import torch.nn as nn

import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable



Criando o <b>R</b>:

Uma distribuição Gaussiana com média 4,0 e desvio-padrão 1,25.

In [91]:
#Criamos a função que é o dataset R, ou seja, a data que vamos copiar:
def get_distribution_sampler(mu, sigma):
    return lambda n: tc.Tensor(np.random.normal(mu, sigma, (1, n))) # Gaussiana

Criando o <b>I</b>:

Uma distribuição uniforme.

In [92]:
#Função do ruído utilizado para gerar entropia

def get_generator_input_sampler():
    return lambda m,n: tc.rand(m,n)



Criando o <b>G</b>:

A nossa rede geradora.

G is going to get the uniformly distributed data samples from I and somehow mimic the normally distributed samples from R.


* O artigo utiliza ELU ao invés de RELU, estamos seguindo esta orientação apenas por simplicidade.

In [93]:
class Generator(nn.Module):
    
    def __init__(self,input_size,hidden_size,output_size):
        #No modelo do pytorch/python vc é obrigado a herdar o init da sua classe pai
        super(Generator,self).__init__()
        self.map1 = nn.Linear(input_size,hidden_size)
        self.map2 = nn.Linear(hidden_size,hidden_size)
        self.map3 = nn.Linear(hidden_size, output_size)
    
    #Testar depois com relu
    def forward(self, x):
        x = F.elu(self.map1(x))
        x = F.sigmoid(self.map2(x))
        return self.map3(x)

    
    
    

Criando o <b>D</b>:

Esta rede pegará samples de "R" (dataset real) ou de "G" (dataset sintético) e vai
ter como saída valores no range de (fake,real) = (0,1).

*Note que as redes tem o mesmo tamanho, mas isso não é necessário.

In [94]:
class Discriminator(nn.Module):
    def __init__(self,input_size, hidden_size, output_size):
        super(Discriminator,self).__init__()
        self.map1 = nn.Linear(input_size,hidden_size)
        self.map2 = nn.Linear(hidden_size,hidden_size)
        self.map3 = nn.Linear(hidden_size, output_size)
        
    def forward(self,x):
        x = F.elu(self.map1(x))
        x = F.sigmoid(self.map2(x))
        x = self.map3(x)
        return(x)
    


# Corpo do treinamento

In [110]:
def train_both_nets(D,G,num_epochs,d_steps,g_steps,print_interval = 100):

    use_gpu = tc.cuda.is_available()

        

    for epoch in range(num_epochs):

        #Treina o detetive
        for d_index in range(d_steps):
            
            D.zero_grad()
            
            # 1A - Treine o Discriminante no dataset Real
            d_real_data = Variable(d_sampler(d_input_size))
            d_real_decision = D(preprocess(d_real_data))
            d_real_error = criterion(tc.clamp(d_real_decision, min=0,max=1), Variable(tc.ones(1))) #ones = true(input_size=d_input_func(d_input_size)
            d_real_error.backward() #Computa os gradientes, mas não muda os parãmetros (não proapga?

            # 1B- Treina o Discriminante no dataset Falso
            d_gen_input = Variable(gi_sampler(minibatch_size, g_input_size))
            d_fake_data = G(d_gen_input).detach()
            d_fake_decision = D(preprocess(d_fake_data.t()))
            d_fake_error = criterion(tc.clamp(d_fake_decision,min=0,max=1), Variable(tc.zeros(1))) #zeros = fake
            d_fake_error.backward()
            d_optimizer.step() #Atualiza os pesos da Discriminant, baseado nos gradientes de 2 erros.

        #Treina o falsificador
        for g_index in range(g_steps):

            # 2. Treine falsificador com as respostas do Detetive, mas não treine D (nesses labels)
            G.zero_grad() #sempre zere os gradientes no inicio do loop
            # O input da rede geradora é o ruído gerador de entropia
            gen_input = Variable(gi_sampler(minibatch_size, g_input_size))
            g_fake_data = G(gen_input)
            #Detetive decide sobre a criação feita pelo falsificador(input_size=d_input_func(d_input_size)
            dg_fake_decision = D(preprocess(g_fake_data.t()))
            g_error = criterion(tc.clamp(dg_fake_decision,min=0,max=1), Variable(tc.ones(1))) #Calcula a perda entre o dito e uma matriz de 1's

            g_error.backward() # Calcula a propagação do erro
            g_optimizer.step() # Atualiza os parâmetros do G's.

        #Print do artigo
        if epoch % print_interval == 0:
            print("%s: D: %s/%s G: %s (Real: %s, Fake: %s) " % (epoch,
                                                                extract(d_real_error)[0],
                                                                extract(d_fake_error)[0],
                                                                extract(g_error)[0],
                                                                stats(extract(d_real_data)),
                                                                stats(extract(d_fake_data))))
    return (D,G)
tats(extract(d_fake_data))))
    return (D,G)

In [96]:
def extract(v):
    return v.data.storage().tolist()

def stats(d):
    return [np.mean(d), np.std(d)]

In [97]:
def decorate_with_diffs(data, exponent):
    mean = tc.mean(data.data, 1, keepdim=True)
    mean_broadcast = tc.mul(tc.ones(data.size()), mean.tolist()[0][0])
    diffs = tc.pow(data - Variable(mean_broadcast), exponent)
    return tc.cat([data, diffs], 1)

# Main part of the program

### Variáveis referentes às distribuições de probabilidade

In [98]:
# Data params
data_mean = 4
data_stddev = 1

In [99]:
### Variáveis referentes às redes generativas DETETIVE e FALSIFICADOR

In [100]:
# Model params
g_input_size = 1     # Random noise dimension coming into generator, per output vector
g_hidden_size = 50   # Generator complexity
g_output_size = 1    # size of generated output vector
d_input_size = 100   # Minibatch size - cardinality of distributions
d_hidden_size = 50   # Discriminator complexity
d_output_size = 1    # Single dimension for 'real' vs. 'fake'
minibatch_size = d_input_size

### Variáveis referentes ao treinamento

In [101]:
d_learning_rate = 2e-4  # 2e-4
g_learning_rate = 2e-4
optim_betas = (0.9, 0.999)
num_epochs = 30000 # Ver se realmente são tantas época aí
d_steps = 1  # 'k' steps in the original GAN paper. Can put the discriminator on higher training freq than generator
g_steps = 1

### Quais dados estão sendo utilizados

In [102]:
# ### Uncomment only one of these
#(name, preprocess, d_input_func) = ("Raw data", lambda data: data, lambda x: x)
(name, preprocess, d_input_func) = ("Data and variances", lambda data: decorate_with_diffs(data, 2.0), lambda x: x * 2)



In [103]:
print("Using data [%s]" % (name))


Using data [Data and variances]


### Programa

In [104]:
# Vamos entender como funciona os samplers!

In [105]:
d_sampler = get_distribution_sampler(data_mean, data_stddev)
gi_sampler = get_generator_input_sampler()


### Modelos

In [106]:
#Istancia das duas redes
G = Generator(input_size=g_input_size, hidden_size=g_hidden_size, output_size=g_output_size)
D = Discriminator(input_size = d_input_func(d_input_size), hidden_size=d_hidden_size, output_size=d_output_size)


In [107]:
criterion = nn.BCELoss()  # Binary cross entropy: http://pytorch.org/docs/nn.html#bceloss
#Adam optimizer para redes discriminativas com parâmetros, lr= 2e-4, Beta=0.9 e gama = 0.999
#Da teoria de otimização contínua, coloque o Beta proximo de 0.9 e vá aumentando gama até antes de divergir
d_optimizer = optim.Adam(D.parameters(), lr=d_learning_rate, betas=optim_betas)
g_optimizer = optim.Adam(G.parameters(), lr=g_learning_rate, betas=optim_betas)

### Treinamento

In [108]:
print_interval = 200

use_gpu = tc.cuda.is_available()

#if use_gpu:
 #   D,G = D.cuda(), G.cuda()
#    print("done")
    

In [111]:
D,G = train_both_nets(D = D, G = G, num_epochs = num_epochs, d_steps = 1, g_steps = 1,print_interval=print_interval)

/home/gabriel/anaconda3/envs/py35/lib/python3.5/site-packages/torch/nn/functional.py:767: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


0: D: 0.05390967056155205/0.1371496468782425 G: 2.0549092292785645 (Real: [4.0366899156570435, 1.0187636422532909], Fake: [0.38112356513738632, 0.0121997835790393]) 
200: D: 0.012576114386320114/-1.000088900582341e-12 G: 27.63102149963379 (Real: [3.9617966091632844, 0.88256202822420626], Fake: [1.8885375034809113, 0.057050503363188135]) 
400: D: -1.000088900582341e-12/-1.000088900582341e-12 G: 27.63102149963379 (Real: [4.0895655095577244, 1.0639598380221824], Fake: [2.0057375431060791, 0.071615221580520139]) 
600: D: -1.000088900582341e-12/-1.000088900582341e-12 G: 27.63102149963379 (Real: [4.0670987522602084, 1.0134667359627552], Fake: [2.1116127848625181, 0.085502138541410824]) 
800: D: -1.000088900582341e-12/-1.000088900582341e-12 G: 27.63102149963379 (Real: [3.8839497321844103, 0.9615737455749056], Fake: [2.2225888419151305, 0.094455737604130202]) 
1000: D: -1.000088900582341e-12/-1.000088900582341e-12 G: 27.63102149963379 (Real: [4.1755913305282597, 0.9328939319415952], Fake: [2.3

9400: D: -1.000088900582341e-12/-1.000088900582341e-12 G: 27.63102149963379 (Real: [3.8222414863109591, 1.0821972872836247], Fake: [2.4665086793899538, 0.13351851882875537]) 
9600: D: -1.000088900582341e-12/-1.000088900582341e-12 G: 27.63102149963379 (Real: [4.1051374828815463, 1.1336771564267185], Fake: [2.4462944817543031, 0.12706320044702435]) 
9800: D: -1.000088900582341e-12/-1.000088900582341e-12 G: 27.63102149963379 (Real: [4.0757796454429629, 0.82678367909228467], Fake: [2.4594511032104491, 0.12923275094156728]) 
10000: D: -1.000088900582341e-12/-1.000088900582341e-12 G: 27.63102149963379 (Real: [4.0745465314388278, 1.0573504737263109], Fake: [2.4287514805793764, 0.13075617616208707]) 
10200: D: -1.000088900582341e-12/-1.000088900582341e-12 G: 27.63102149963379 (Real: [4.0141352319717409, 1.0398673582364173], Fake: [2.4433907866477966, 0.1263448505472671]) 
10400: D: -1.000088900582341e-12/-1.000088900582341e-12 G: 27.63102149963379 (Real: [3.9172781538963317, 0.8415341467201997

18800: D: -1.000088900582341e-12/-1.000088900582341e-12 G: 27.63102149963379 (Real: [4.0789195454120639, 0.89288775450831437], Fake: [2.4602111148834229, 0.12163163747831909]) 
19000: D: -1.000088900582341e-12/-1.000088900582341e-12 G: 27.63102149963379 (Real: [3.9242075645923613, 0.95835705191235632], Fake: [2.4560175275802614, 0.12534866552335514]) 
19200: D: -1.000088900582341e-12/-1.000088900582341e-12 G: 27.63102149963379 (Real: [3.9301237285137178, 0.99231781899445359], Fake: [2.4757011055946352, 0.11895233447998119]) 
19400: D: -1.000088900582341e-12/-1.000088900582341e-12 G: 27.63102149963379 (Real: [4.0061802864074707, 1.0465876831770151], Fake: [2.454839165210724, 0.1288185463080167]) 
19600: D: -1.000088900582341e-12/-1.000088900582341e-12 G: 27.63102149963379 (Real: [4.0751066064834598, 1.0476726836377503], Fake: [2.4617360687255858, 0.12182696922923031]) 
19800: D: -1.000088900582341e-12/-1.000088900582341e-12 G: 27.63102149963379 (Real: [3.9532866311073303, 1.052429122831

28200: D: -1.000088900582341e-12/-1.000088900582341e-12 G: 27.63102149963379 (Real: [3.8464529645442962, 0.97004526445354755], Fake: [2.4385418915748596, 0.12083711460775341]) 
28400: D: -1.000088900582341e-12/-1.000088900582341e-12 G: 27.63102149963379 (Real: [4.1675326001644137, 1.0427119434222172], Fake: [2.4712990736961364, 0.12340838615604922]) 
28600: D: -1.000088900582341e-12/-1.000088900582341e-12 G: 27.63102149963379 (Real: [4.1578790473937985, 0.92090345940334983], Fake: [2.4790980982780457, 0.12050110072731238]) 
28800: D: -1.000088900582341e-12/-1.000088900582341e-12 G: 27.63102149963379 (Real: [4.2964280200004579, 1.1383489412357672], Fake: [2.4466008687019349, 0.12158073094076159]) 
29000: D: -1.000088900582341e-12/-1.000088900582341e-12 G: 27.63102149963379 (Real: [3.9353268301486968, 1.063629898288184], Fake: [2.4760322380065918, 0.11785899482886121]) 
29200: D: -1.000088900582341e-12/-1.000088900582341e-12 G: 27.63102149963379 (Real: [3.9920670342445375, 0.937966334753

# Solution

 Entendendo o código

Na parte '1A' do treino do detetive, jogamos (foward step) para o detetive dados reais e dados fakes, afim de achar os 2 erros com o criterion de custo. O método backward() calcula os gradientes no grafo de computação e vai ser guardado para atualizar os pesos da rede, feito no método d_optimizer.step(). 

*OBS: Note que G é utilizado apenas para fazer predict ali, por tanto não é treinado.

Já na parte '2A', que é o treinamento do falsificador, criamos uma tela falsa (g_fake_data) e damos para o detetive decidir o que acha da falsificação (dg_fake_decision) para só depois calcularmos o erro do critério de custo (d_fake_error). Após isso calculamos os gradientes do grafo de computação e utilizamos para atualizar os pesos no g_optimizer.step().
    